In [49]:
from PIL import Image, ImageSequence
import numpy as np
import functools
import kgpy.img.coalignment.image_coalignment as img_align
from numba import njit
from skimage.transform import rotate
import scipy.ndimage


# Image Rotation in Python
Start first by testing out three different rotation codes and compare their speeds.

In [50]:
a = np.random.rand(500, 500)



In [51]:

def pillow_rotate(a,angle):
    if a.ndim == 2:
        im = Image.fromarray(a)
        im.rotate(angle,Image.BILINEAR)
        a = np.asarray(im)
    else:    
        b = np.moveaxis(a,-1,0)
        for i,img in enumerate(b):
            im = Image.fromarray(img)
            im.rotate(angle,Image.BILINEAR)
            a[...,i] = np.asarray(im)

    return a


In [52]:
ang = 10

# %timeit pillow_rotate(a,ang)
# %timeit rotate(a, ang, order=1)
# %timeit rotate(a, ang, order=3)
# %timeit scipy.ndimage.rotate(a,ang,prefilter = False, order = 1)
# %timeit scipy.ndimage.rotate(a,ang)


Since MART (for ESIS) relies heavily on repeated rotation calls let's test breaking up parts of the rotation and include
some "memoisation" (Thanks Roy)

In [53]:
@functools.lru_cache(maxsize = 1)
def coord_rotate(img_shp,angle,origin):

    x, y = np.array(np.meshgrid(np.arange(img_shp[0]), np.arange(img_shp[1])))
    c = np.cos(angle)
    s = np.sin(angle)
    x_prime = c*(x-origin[0]) + s*(y-origin[1])
    y_prime = -s*(x-origin[0]) + c*(y-origin[1])
    return np.array([x_prime+origin[0],y_prime+origin[1]])

In [54]:
origin = np.array([a.shape[0]//2,a.shape[1]//2])
prime_coords = coord_rotate(a.shape,ang,origin)


TypeError: unhashable type: 'numpy.ndarray'